In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Пути к файлам
labels_csv='../input/dog-breed-identification/labels.csv'
sample_submission_csv='../input/dog-breed-identification/sample_submission.csv'

# Пути до картинок
jpg_train='../input/dog-breed-identification/train/{}.jpg'  
jpg_test='../input/dog-breed-identification/test/{}.jpg'

# Настройки нейросети
im_resize = 64 # Размер изображения
num_class = 120 # Кол-во классов
batch_size = 32
Epochs = 50


In [3]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow.keras as keras
from keras import regularizers
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.metrics import categorical_accuracy, categorical_crossentropy
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator


2025-08-18 03:53:41.268388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755489221.280453       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755489221.284487       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 03:53:41.299848: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras.layers.convolutional'

In [4]:
def gen_graph(history, title):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('crossentropy ' + title)
    plt.ylabel('crossentropy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('categorical_accuracy ' + title)
    plt.ylabel('categorical_accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()


In [5]:
df_train = pd.read_csv(labels_csv) 
df_test = pd.read_csv(sample_submission_csv)


In [6]:
df_train.head()


,id,breed
0,8406d837b2d7fac1c3cd621abb4c4f9e,west_highland_white_terrier
1,e270622b5ffec8294d7e7628c4ff6c1e,brittany_spaniel
2,41295c36303043fc587e791b14ef2272,basset
3,b63b0200ddbb97df81972b26574959ab,boxer
4,2c64e362c9aa29450082291264dcba29,flat-coated_retriever


In [7]:
df_test.head()


,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,9f68d045a396679a778eb54c5ed29038,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,f375e6363bc21dcd3cb65637c7855e9c,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,010e87fdf252645a827e37470e65e842,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,ad2dfa0202d8ea3766fea1e743cd5166,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,a579a1802c57cfbc31b79781f6f37a39,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [8]:
labels = df_train['breed']
one_hot = pd.get_dummies(labels, sparse = True)
one_hot_labels = np.asarray(one_hot)
#one_hot_labels


In [9]:
x_train = []
y_train = []
x_test = []


In [10]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = load_img(jpg_train.format(f), target_size=(im_resize, im_resize))
    img_resized = img_to_array(img)
    x_train.append(img_resized)
    label = one_hot_labels[i]
    y_train.append(label)
    i += 1


NameError: name 'tqdm' is not defined

In [11]:
for f in tqdm(df_test['id'].values):
    img = load_img(jpg_test.format(f), target_size=(im_resize, im_resize))
    img_resized = img_to_array(img)
    x_test.append(img_resized)


NameError: name 'tqdm' is not defined

In [12]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, shuffle=True,  test_size=0.2)


NameError: name 'train_test_split' is not defined

In [13]:
del x_train, y_train, df_train


In [14]:
train_datagen = ImageDataGenerator(
                            rescale=1./255 #нормализация
                            )

test_datagen = ImageDataGenerator(rescale=1./255)

# Создаем генераторы 
train_generator =train_datagen.flow(np.array(X_train), np.array(Y_train), 
                               batch_size=batch_size)
test_generator =test_datagen.flow(np.array(X_valid), np.array(Y_valid),
                              batch_size=batch_size*5)


NameError: name 'ImageDataGenerator' is not defined

In [15]:
# Создаем последовательную модель
model = Sequential()

# сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(im_resize, im_resize, 3), activation='relu'))
# первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))



# сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))




# сверточный слой
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# Третий слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))



# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))


# Выходной полносвязный слой
model.add(Dense(num_class, activation='softmax'))


NameError: name 'Conv2D' is not defined

In [16]:
model.compile(optimizer='adam',
          loss='categorical_crossentropy', 
           metrics=[categorical_accuracy])


NameError: name 'categorical_accuracy' is not defined

In [17]:
print(model.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [18]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop=EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

checkpoint_callback = ModelCheckpoint('model_best.hdf5',
                                      monitor='val_categorical_accuracy', 
                                      save_best_only=True,
                                      verbose=1)


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=model_best.hdf5

In [19]:
history = model.fit(
    train_generator,
    callbacks=[earlystop, checkpoint_callback],
    epochs=Epochs,
    steps_per_epoch=len(train_generator),
    
    validation_data=test_generator,
    validation_steps=len(test_generator))


NameError: name 'train_generator' is not defined

In [20]:
#График точности на валидационной и обучающей выборке
gen_graph(history,
          "график точности")


NameError: name 'history' is not defined

In [21]:
from tensorflow.keras.models import load_model
model=load_model("model_best.hdf5")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'model_best.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [22]:
preds = model.predict(np.array(x_test))


2025-08-18 03:53:47.489659: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-08-18 03:53:47.551751: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


ValueError: math domain error

In [23]:
sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)


NameError: name 'preds' is not defined

In [24]:
sub.to_csv("output_rmsprop_aug.csv", index=False)


NameError: name 'sub' is not defined